In [1]:
# Load API key.
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
# First we initialize the model we want to use.
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)


# For this tutorial we will use custom tool that returns pre-defined values for weather in two cities (NYC & SF)
from typing import Literal

from langchain_core.tools import tool


@tool
def get_weather(location: str):
    """Use this to get weather information from a given location."""
    if location.lower() in ["nyc", "new york"]:
        return "It might be cloudy in nyc"
    elif location.lower() in ["sf", "san francisco"]:
        return "It's always sunny in sf"
    else:
        raise AssertionError("Unknown Location")


tools = [get_weather]

# We need a checkpointer to enable human-in-the-loop patterns
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

# Define the graph

from langgraph.prebuilt import create_react_agent

graph = create_react_agent(
    model, tools=tools, interrupt_before=["tools"], checkpointer=memory
)

In [16]:
def print_stream(stream):
    """A utility to pretty print the stream."""
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [17]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "42"}}
inputs = {"messages": [("user", "what is the weather in SF, CA?")]}

print_stream(graph.stream(inputs, config, stream_mode="values"))

================================ Human Message =================================

what is the weather in SF, CA?


================================== Ai Message ==================================
Tool Calls:
  get_weather (call_WkaB4wSkdD29j0SWYt9SYLdQ)
 Call ID: call_WkaB4wSkdD29j0SWYt9SYLdQ
  Args:
    location: SF, CA


In [18]:
snapshot = graph.get_state(config)
print("Next step: ", snapshot.next)

Next step:  ('tools',)


In [19]:
print_stream(graph.stream(None, config, stream_mode="values"))

================================== Ai Message ==================================
Tool Calls:
  get_weather (call_WkaB4wSkdD29j0SWYt9SYLdQ)
 Call ID: call_WkaB4wSkdD29j0SWYt9SYLdQ
  Args:
    location: SF, CA
================================= Tool Message =================================
Name: get_weather

Error: AssertionError('Unknown Location')
 Please fix your mistakes.
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_r4DqyrbqrfOw4mVcczs1c3Pm)
 Call ID: call_r4DqyrbqrfOw4mVcczs1c3Pm
  Args:
    location: San Francisco, CA


In [20]:
state = graph.get_state(config)

last_message = state.values["messages"][-1]
last_message.tool_calls[0]["args"] = {"location": "San Francisco"}

graph.update_state(config, {"messages": [last_message]})

{'configurable': {'thread_id': '42',
  'checkpoint_ns': '',
  'checkpoint_id': '1f005e8f-0eb6-658c-8004-a2176749b048'}}

In [21]:
last_message

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_r4DqyrbqrfOw4mVcczs1c3Pm', 'function': {'arguments': '{"location":"San Francisco, CA"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 94, 'total_tokens': 112, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6ec83003ad', 'id': 'chatcmpl-BDKGEcvHCyWLyphEtJRaGxCbrbpF9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-63ce9e3d-864e-4a05-87cc-352d5099e7f5-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'San Francisco'}, 'id': 'call_r4DqyrbqrfOw4mVcczs1c3Pm', 'type': 'tool_call'}], usage_metadata={'input_tokens': 94, 'output_tokens': 18, 'total_tokens': 112, 'input_token_details': {'audio': 0

In [22]:
print_stream(graph.stream(None, config, stream_mode="values"))

================================== Ai Message ==================================
Tool Calls:
  get_weather (call_r4DqyrbqrfOw4mVcczs1c3Pm)
 Call ID: call_r4DqyrbqrfOw4mVcczs1c3Pm
  Args:
    location: San Francisco
================================= Tool Message =================================
Name: get_weather

It's always sunny in sf
================================== Ai Message ==================================

The weather in San Francisco is always sunny.
